In [ ]:
# Import required libraries
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Scikit-learn for evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print("✅ Libraries imported successfully!")

## 1. Load Preprocessed Data

In [ ]:
# Load preprocessed data
X_train = np.load('../models/X_train.npy')
X_test = np.load('../models/X_test.npy')
y_train = np.load('../models/y_train.npy')
y_test = np.load('../models/y_test.npy')

print("Data loaded successfully!")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Load tokenizer and label encoder
with open('../models/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

with open('../models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

print("✅ Tokenizer and label encoder loaded!")
print(f"\nVocabulary size: {len(tokenizer.word_index) + 1}")
print(f"Number of classes: {len(label_encoder.classes_)}")
print(f"\nJob categories: {list(label_encoder.classes_)}")

## 2. Prepare Data for Training

In [ ]:
# Convert labels to categorical (one-hot encoding)
num_classes = len(label_encoder.classes_)
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

print(f"✅ Labels converted to categorical format!")
print(f"y_train_cat shape: {y_train_cat.shape}")
print(f"y_test_cat shape: {y_test_cat.shape}")
print(f"\nSample one-hot encoded label:")
print(f"Original label: {y_train[0]}")
print(f"One-hot encoded: {y_train_cat[0]}")

In [ ]:
# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
max_length = X_train.shape[1]
embedding_dim = 128
lstm_units = 64

print("Model Parameters:")
print(f"Vocabulary size: {vocab_size}")
print(f"Max sequence length: {max_length}")
print(f"Embedding dimension: {embedding_dim}")
print(f"LSTM units: {lstm_units}")
print(f"Number of classes: {num_classes}")

## 3. Build Bi-LSTM Model

In [ ]:
# Build the model
model = Sequential([
    # Embedding layer
    Embedding(input_dim=vocab_size, 
              output_dim=embedding_dim, 
              input_length=max_length,
              name='embedding'),
    
    # Bidirectional LSTM layer
    Bidirectional(LSTM(lstm_units, return_sequences=False), name='bi_lstm'),
    
    # Dropout for regularization
    Dropout(0.5, name='dropout'),
    
    # Dense layer with ReLU activation
    Dense(64, activation='relu', name='dense_relu'),
    
    # Output layer with Softmax activation
    Dense(num_classes, activation='softmax', name='output')
])

print("✅ Bi-LSTM model built successfully!")
print("\nModel Architecture:")
model.summary()

## 4. Compile Model

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled successfully!")
print("\nCompilation details:")
print(f"Optimizer: Adam")
print(f"Loss function: Categorical Crossentropy")
print(f"Metrics: Accuracy")

## 5. Set up Callbacks

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    '../models/resume_classifier.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

callbacks = [early_stopping, model_checkpoint]

print("✅ Callbacks configured!")
print("- Early Stopping: Monitors val_loss with patience=3")
print("- Model Checkpoint: Saves best model based on val_accuracy")

## 6. Train the Model

In [ ]:
# Train the model
print("🚀 Starting model training...\n")

epochs = 15
batch_size = 4

history = model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Model training complete!")

## 7. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', marker='o')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', marker='o')
axes[1].plot(history.history['val_loss'], label='Validation Loss', marker='s')
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training history plots saved to results/training_history.png")

## 8. Evaluate Model on Test Set

In [ ]:
# Evaluate on test set
print("📊 Evaluating model on test set...\n")

test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

In [ ]:
# Get predictions
y_pred_probs = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nSample Predictions:")
for i in range(min(5, len(y_test))):
    true_label = label_encoder.classes_[y_test[i]]
    pred_label = label_encoder.classes_[y_pred[i]]
    confidence = y_pred_probs[i][y_pred[i]] * 100
    
    match = "✓" if true_label == pred_label else "✗"
    print(f"{match} True: {true_label:20s} | Predicted: {pred_label:20s} | Confidence: {confidence:.2f}%")

## 9. Classification Report

In [ ]:
# Print classification report
print("\nClassification Report:")
print("="*80)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))

## 10. Confusion Matrix

In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Resume Classification', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('../results/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrix saved to results/confusion_matrix.png")

## 11. Save Model

In [ ]:
# Save the final model
model.save('../models/resume_classifier.h5')
print("✅ Model saved to models/resume_classifier.h5")

# Also save in Keras format
model.save('../models/resume_classifier.keras')
print("✅ Model saved to models/resume_classifier.keras")

## 12. Save Training Metrics

In [ ]:
# Get final training metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
epochs_trained = len(history.history['accuracy'])

# Create metrics text file
metrics_text = f"""TRAINING METRICS - DAY 3
====================================

Model Information:
- Model Type: Bidirectional LSTM
- Architecture: Embedding → Bi-LSTM → Dropout → Dense (ReLU) → Dense (Softmax)
- Embedding Dimension: {embedding_dim}
- LSTM Units: {lstm_units}
- Total Parameters: {model.count_params():,}

Training Configuration:
- Optimizer: Adam
- Loss Function: Categorical Crossentropy
- Batch Size: {batch_size}
- Epochs Trained: {epochs_trained}/{epochs}
- Validation Split: 20%

Dataset:
- Total Samples: {len(X_train) + len(X_test)}
- Training Samples: {len(X_train)}
- Testing Samples: {len(X_test)}
- Number of Classes: {num_classes}
- Vocabulary Size: {vocab_size}
- Max Sequence Length: {max_length}

Training Results:
- Final Training Accuracy: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)
- Final Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)
- Final Training Loss: {final_train_loss:.4f}
- Final Validation Loss: {final_val_loss:.4f}

Test Results:
- Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)
- Test Loss: {test_loss:.4f}

Model Files:
- models/resume_classifier.h5
- models/resume_classifier.keras
- models/tokenizer.pkl
- models/label_encoder.pkl

Visualization Files:
- results/training_history.png
- results/confusion_matrix.png

Job Categories:
{chr(10).join(f'  {i+1}. {cat}' for i, cat in enumerate(label_encoder.classes_))}

Ready for Day 4: Testing & Deployment!
"""

with open('../results/metrics.txt', 'w') as f:
    f.write(metrics_text)

print("✅ Metrics saved to results/metrics.txt")
print("\n" + metrics_text)

## 13. Summary

In [ ]:
# Print final summary
print("\n" + "="*80)
print("DAY 3 SUMMARY - DEEP LEARNING MODEL TRAINING")
print("="*80)

print(f"\n✅ Model Architecture: Bi-LSTM")
print(f"   - Layers: Embedding → Bi-LSTM → Dropout → Dense → Output")
print(f"   - Total Parameters: {model.count_params():,}")

print(f"\n✅ Training Complete:")
print(f"   - Epochs: {epochs_trained}/{epochs}")
print(f"   - Final Training Accuracy: {final_train_acc*100:.2f}%")
print(f"   - Final Validation Accuracy: {final_val_acc*100:.2f}%")

print(f"\n✅ Test Performance:")
print(f"   - Test Accuracy: {test_accuracy*100:.2f}%")
print(f"   - Test Loss: {test_loss:.4f}")

print(f"\n✅ Files Saved:")
print(f"   - Model: models/resume_classifier.h5, models/resume_classifier.keras")
print(f"   - Metrics: results/metrics.txt")
print(f"   - Plots: results/training_history.png, results/confusion_matrix.png")

print(f"\n🚀 Ready for Day 4: Testing & Deployment!")
print("="*80)